In [6]:
from transformers import pipeline
import nltk
import pandas as pd
import numpy as np
import os
import re

from nltk.corpus import stopwords
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment import SentimentAnalyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\remes\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\remes\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\remes\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
summarizer = pipeline("summarization", truncation=True)
analyzer = SentimentIntensityAnalyzer()
token_analyzer = SentimentAnalyzer()

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [4]:
def analyze_sentiment(text):
    sentiment = analyzer.polarity_scores(text)
    return sentiment


def article_summarizer(article):
    summary = summarizer(article, max_length=300, min_length=100, do_sample=False)
    
    return summary[0]['summary_text']

def get_companies_names():
    db = next(get_db())
    try:
        stocks = db.query(Stock.title).all()
        return stocks
    finally:
        db.close()

___Basic version of the algorithm___

Analyzes pre-processed text. Processing includes: removing advertisements, removing unnecessary abbreviations and abbreviations, repeating the same thing.

In [6]:
articles_marks = []
companies_names = list((i[0] for i in get_companies_names()))

for root, dirs, files in os.walk(os.getcwd() + "/data/stocks"):
    for dir_name, company_name in zip(dirs, companies_names):
        folder_path = os.path.join(root, dir_name)
        pos_mark = 0
        neg_mark = 0
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            article_file = open(file_path, 'r', encoding="utf-8")
            article_text = article_file.read()
            sum_article = article_summarizer(article_text)
            analyze = analyze_sentiment(sum_article)
            if analyze['pos'] > analyze['neg']:
                pos_mark += 1
            else:
                neg_mark += 1
        articles_marks.append({'stock': company_name, 'positives': pos_mark, 'negatives': neg_mark})
        break
base_articles_df = pd.DataFrame(articles_marks, columns=['stock', 'positives', 'negatives'])
print(base_articles_df)

Your max_length is set to 300, but your input_length is only 83. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=41)
Your max_length is set to 300, but your input_length is only 230. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=115)


                stock  positives  negatives
0  NVIDIA Corporation          3          0


__Adding news sentiment scores for each stock to the database.__

In [2]:
from sqlalchemy import text
from database.models import Stock, StockSentiment
from database.db import get_db

def add_to_db(df):
    db = next(get_db())
    try:
        db.query(StockSentiment).delete()
        db.commit()

        db.execute(text("ALTER SEQUENCE stock_sentiments_id_seq RESTART WITH 1"))
        db.commit()
        
        print("Updating stock sentiments...")

        stock_sentiments = []
        for index, row in df.iterrows():
            stock = db.query(Stock).filter_by(title=row['stock']).first()
            if stock:
                stock_sentiment = StockSentiment(
                    stock_id=stock.id,
                    positives=row['positives'],
                    negatives=row['negatives']
                )
                stock_sentiments.append(stock_sentiment)
            else:
                print(f"Stock '{row['stock']}' not found in the Stock table")

        db.bulk_save_objects(stock_sentiments)
        db.commit()

        print("Stock sentiments updated successfully.")
    except Exception as e:
        db.rollback()
        print(f"Failed to update stock sentiments: {e}")
    finally:
        db.close()
        
    
def clear_db():
    db = next(get_db())
    try:
        db.query(StockSentiment).delete()
        db.commit()
        print("Database cleared successfully.")
    except Exception as e:
        db.rollback()
        print(f"Failed to update stock sentiments: {e}")
    finally:
        db.close()

In [7]:
add_to_db(base_articles_df)

Updating stock sentiments...
Stock sentiments updated successfully.


The second version of the algorithm using tokenization.

In [9]:
articles_marks = []
companies_names = list((i[0] for i in get_companies_names()))
stop_words = set(stopwords.words("english"))
articles_list = []

def delete_punctuation(article):
    pattern = r'[^\w\s]'
    article = re.sub(pattern,'',article)
    return article


def sentence_tokenization(sentence):
    article_word_list = nltk.tokenize.word_tokenize(sentence)
    return article_word_list


def article_tokenization(article):
    sentences = nltk.tokenize.sent_tokenize(article)
    return sentences


def delete_stop_words(article_list):
    global stop_words
    processed_article = [word for word in article_list if word.lower() not in stop_words]
    return processed_article


def analyze_tokenized_sentiment(tokenized_article):
    sentiment_scores = [token_analyzer.all_words(word) for word in tokenized_article if word.isalpha()]
    return sentiment_scores

for root, dirs, files in os.walk(os.getcwd() + "/data/stocks"):
    for dir_name, company_name in zip(dirs, companies_names):
        folder_path = os.path.join(root, dir_name)
        pos_mark = 0
        neg_mark = 0
        articles_list = []
        for filename in os.listdir(folder_path):
            words_list = []
            sentences_list = []
            file_path = os.path.join(folder_path, filename)
            article_file = open(file_path, 'r', encoding="utf-8")
            article_text = article_file.read()
            article = re.sub(r'\.(?!\s)', '. ', article_text)
            sentences = article_tokenization(article_text)
            for sentence in sentences:
                sentence = delete_punctuation(sentence)
                tokenized_sentence = sentence_tokenization(sentence)
                without_base_sw = delete_stop_words(tokenized_sentence)
                words_list.append(without_base_sw)
            for word_list in words_list:
                sentence = ' '.join(word_list)
                
                sentence = sentence.capitalize()
                
                sentence = re.sub(r'([.!?])(\w)', r'\1 \2', sentence)
                sentences_list.append(sentence)
            articles_list.append(sentences_list)
        pos_counter = 0 
        neg_counter = 0
        for article in articles_list:
            compound_counter: float = 0
            for sentence in article:
                if analyze_sentiment(sentence)["neg"] > analyze_sentiment(sentence)["pos"]:
                    neg_counter += 1
                elif analyze_sentiment(sentence)["pos"] > analyze_sentiment(sentence)["neg"]:
                    pos_counter += 1
        articles_marks.append({'stock': company_name, 'positives': pos_counter, 'negatives': neg_counter})
    

# tokenized_sentences_df = pd.DataFrame(articles_marks, columns=['stock', 'positives', 'negatives'])
# tokenized_sentences_df

,stock,positives,negatives
0,NVIDIA Corporation,154,60
1,Palantir Technologies,126,26
2,Intel Corporation,148,66
3,Lumen Technologies,178,44
4,Tesla,141,47
5,Ford Motor Company,113,23
6,Apple,63,12
7,Warner Bros. Discovery,93,27
8,Advanced Micro Devices,140,24
9,Nu Holdings Ltd.,91,9


In [10]:
add_to_db(tokenized_sentences_df)

Updating stock sentiments...
Stock sentiments updated successfully.


In [ ]:
db = next(get_db())
db.query(StockSentiment).delete()
db.commit()

In [35]:
articles_marks = []
companies_names = list((i[0] for i in get_companies_names()))
stop_words = set(stopwords.words("english"))
articles_list = []

for root, dirs, files in os.walk(os.getcwd() + "/data/stocks"):
    for dir_name, company_name in zip(dirs, companies_names):
        folder_path = os.path.join(root, dir_name)
        pos_mark = 0
        neg_mark = 0
        articles_list = []
        for filename in os.listdir(folder_path):
            words_list = []
            sentences_list = []
            file_path = os.path.join(folder_path, filename)
            article_file = open(file_path, 'r', encoding="utf-8")
            article_text = article_file.read()
            article = re.sub(r'\.(?!\s)', '. ', article_text)
            sentences = article_tokenization(article_text)
            for sentence in sentences:
                sentence = delete_punctuation(sentence)
                tokenized_sentence = sentence_tokenization(sentence)
                without_base_sw = delete_stop_words(tokenized_sentence)
                words_list.append(without_base_sw)
            for word_list in words_list:
                sentence = ' '.join(word_list)
                
                sentence = sentence.capitalize()
                sentence = sentence + "."
                
                sentences_list.append(sentence)
            articles_list.append(sentences_list)
        pos_counter = 0 
        neg_counter = 0
        full_articles_list = []
        for article in articles_list:
            full_article = ""
            for sentence in article:
                full_article = full_article + sentence + " "
            full_articles_list.append(full_article)
            
        for article_text in full_articles_list:
            if analyze_sentiment(article_text)["compound"] <= 0:
                neg_counter += 1
            elif analyze_sentiment(article_text)["compound"] > 0:
                pos_counter += 1
            print(analyze_sentiment(article_text))
        articles_marks.append({'stock': company_name, 'positives': pos_counter, 'negatives': neg_counter})
        
tokenized_articles_df = pd.DataFrame(articles_marks, columns=['stock', 'positives', 'negatives'])
tokenized_articles_df

{'neg': 0.059, 'neu': 0.859, 'pos': 0.082, 'compound': 0.9186}
{'neg': 0.069, 'neu': 0.812, 'pos': 0.119, 'compound': 0.9887}
{'neg': 0.051, 'neu': 0.861, 'pos': 0.087, 'compound': 0.8467}
{'neg': 0.069, 'neu': 0.817, 'pos': 0.115, 'compound': 0.967}
{'neg': 0.028, 'neu': 0.777, 'pos': 0.195, 'compound': 0.9971}
{'neg': 0.059, 'neu': 0.832, 'pos': 0.109, 'compound': 0.9363}
{'neg': 0.056, 'neu': 0.764, 'pos': 0.18, 'compound': 0.9952}
{'neg': 0.028, 'neu': 0.773, 'pos': 0.198, 'compound': 0.9956}
{'neg': 0.061, 'neu': 0.777, 'pos': 0.162, 'compound': 0.9851}
{'neg': 0.033, 'neu': 0.811, 'pos': 0.156, 'compound': 0.9783}
{'neg': 0.01, 'neu': 0.812, 'pos': 0.178, 'compound': 0.9916}
{'neg': 0.028, 'neu': 0.773, 'pos': 0.198, 'compound': 0.9956}
{'neg': 0.037, 'neu': 0.827, 'pos': 0.136, 'compound': 0.9843}
{'neg': 0.053, 'neu': 0.69, 'pos': 0.257, 'compound': 0.9963}
{'neg': 0.043, 'neu': 0.787, 'pos': 0.17, 'compound': 0.9966}
{'neg': 0.054, 'neu': 0.759, 'pos': 0.187, 'compound': 0.982

,stock,positives,negatives
0,Tesla,6,0
1,NVIDIA,5,0
2,Alphabet A,5,0
3,CrowdStrike Holdings,5,0
4,Apple,4,0
5,Amazon.com,7,0
6,Verizon,7,0
7,AMC Entertainment,5,0
8,Alphabet C,8,0
9,Coinbase Global,4,0


In [3]:
clear_db()

Database cleared successfully.
